In [1]:
import sys
import random
import math

In [2]:
def gibbs_sampler(dna: list[str], k: int, t: int, N: int) -> list[str]:    
    finalScore = 10000000000000.0
    allDNA = dna
    for round in range(20):

        motifs = []
        for seq in allDNA:
            motifs.append(random_motif(seq, k))
        bestMotifs = motifs.copy()

        for j in range(20):
            i = random.randint(0, t) - 1
            motifi = motifs[i]
            dnai = dna[i]

            del motifs[i]
            del dna[i]
            
            profile = create_profile(motifs, k)
            motifs = find_motifs(profile, dna, k)

            motifi = random_motif(dnai, k)
            motifs.insert(i, motifi)
            dna.insert(i, dnai)

            motifsScore = calculate_set_score(motifs)
            bestMotifsScore = calculate_set_score(bestMotifs)

            if motifsScore < bestMotifsScore: 
                bestMotifs =  motifs.copy()
                bestMotifsScore = motifsScore

        if bestMotifsScore < finalScore:
            finalMotifs = bestMotifs
            finalScore = bestMotifsScore

    return finalMotifs


def random_motif(seq, k):
    start = random.randint(0, len(seq)-k)
    kmer = seq[start:start+k]
    return kmer

def create_profile(motifs, k):
    profile = []
    for i in range(k):
        countDict = {"A":1, "T":1, "C":1, "G":1}
        for motif in motifs:
            nuc = motif[i]
            countDict[nuc] += 1
        countDict = {key: value / (sum(countDict.values())) for key, value in countDict.items()}
        profile.append(countDict)
    return profile

def calculate_set_score(motifs):
    nucCount = []
    for i in range(len(motifs[0])):
        dict = {"A":0, "T":0, "C":0, "G":0}
        for j in range(len(motifs)):
            dict[motifs[j][i]] += 1
        nucCount.append(dict)

    consensus = 0.0
    for i in nucCount:
        nuc = max(i, key=i.get)
        consensus += (sum(i.values()) - i[nuc])
    return consensus

def find_motifs(profile, dna, k):
    motifs = []
    for seq in dna:
        text_length = len(seq)
        bestKmer = ""
        bestScore = 0.0
        for i in range(text_length+1-k):
            kmer = seq[i:i+k]
            score = calculate_score(kmer, profile)
            if score > bestScore: ###
                bestScore = score 
                bestKmer = kmer
        motifs.append(bestKmer)
    return motifs

def calculate_score(motif, profile):
    score = 1.0
    for i in range(len(motif)):
        nuc = motif[i]
        score *= profile[i][nuc]
    return score

In [3]:
k = 8 
t = 5
N = 100
dna = ["CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA", "GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG", "TAGTACCGAGACCGAAAGAAGTATACAGGCGT", "TAGATCAAGTTTCAGGTGCACGTCGGTGAACCAA", "TCCACCAGCTCCACGTGCAATGTTGGCCTA"]
correctMotifs = ["AACGGCCA", "AAGTGCCA", "TAGTACCG", "AAGTTTCA", "ACGTGCAA"]
myMotifs = gibbs_sampler(dna, k, t, N)
print("Correct: ", correctMotifs)
print("Mine: ", myMotifs)

calculate_set_score(correctMotifs)

Correct:  ['AACGGCCA', 'AAGTGCCA', 'TAGTACCG', 'AAGTTTCA', 'ACGTGCAA']
Mine:  ['GTGTTCAG', 'GTGCCAAG', 'GTATACAG', 'GTTTCAGG', 'GTGCAATG']


9.0